In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from mgwr.gwr import GWR, MGWR, GWRResults
from mgwr.sel_bw import Sel_BW
from scipy import stats
import matplotlib.pyplot as plt



import seaborn as sns

import geopandas as gpd
%matplotlib inline
sns.set(color_codes=True)
from matplotlib_scalebar.scalebar import ScaleBar

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


gsv = pd.read_csv('./data/gsv_indicators_summarized_county.csv')
zipcode = pd.read_csv('./data/zip_code_database.csv')
census = pd.read_csv('./data/summary_zipcode_oct_1_2020.csv')

df = pd.read_csv('./data/ACSST1Y2021.S2504-Data.csv')

In [ ]:
gsv = gsv.apply(lambda x: x * 100) # optimize code later
gsv['county'] = gsv.county//100

In [ ]:
not_residential = df[['Geography','Estimate Percent occupied housing units Occupied housing units UNITS IN STRUCTURE 10 or more apartments']]

In [ ]:
not_residential.rename(columns={'Estimate Percent occupied housing units Occupied housing units UNITS IN STRUCTURE 10 or more apartments':'apartments', 'Geography':'County'}, inplace=True)

In [ ]:
not_residential['County'] = not_residential['County'].str.split('S', expand=True)[1]
not_residential['County'] = not_residential['County'].astype(int)

# not_residential.head()

In [ ]:
# Merge df with GSV indicators

merged = gsv.merge(not_residential, left_on='county', right_on='County', how='inner')

In [ ]:
import geopandas as gpd
from geopandas import GeoDataFrame

counties = gpd.read_file('https://cdn.jsdelivr.net/npm/us-atlas@3/counties-10m.json')\
                                                                            .sort_values(by='id').reset_index(drop=True)

empty = counties['geometry'].is_empty
counties_non_empty = counties[~empty]

In [ ]:
counties_non_empty['lon'] = counties_non_empty['geometry'].centroid.x
counties_non_empty['lat'] = counties_non_empty['geometry'].centroid.y
counties_non_empty['id'] = counties_non_empty['id'].astype('int')

In [ ]:
data = merged.merge(counties_non_empty, left_on='county', right_on='id', how='inner')

In [ ]:
data.drop(['County', 'id', 'name'], axis=1, inplace=True)

In [ ]:
data['apartments'] = data['apartments'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
data = data.dropna()

In [ ]:
g_y = data['prop_sidewalk'].values.reshape((-1,1))
g_X = data[['apartments']].values

u = data['lon']
v = data['lat']

g_coords = list(zip(u,v))

# data.head(2)

In [ ]:
# gdf = GeoDataFrame(data, crs="EPSG:4326")  # change the EPSG, see whether the result changes

# # gdf.plot(figsize=(50, 50))

# fig, ax = plt.subplots(figsize=(30,30))
# gdf.plot(ax=ax, color='tomato')
# minx, miny, maxx, maxy = gdf.total_bounds
# ax.set_xlim(minx, maxx)
# ax.set_ylim(miny, maxy)

In [ ]:
gwr_selector_o = Sel_BW(g_coords, g_y, g_X)
gwr_bw_o = gwr_selector_o.search()
print(gwr_bw_o)
gwr_results_o = GWR(g_coords, g_y, g_X, gwr_bw_o, constant=True, lwcc=False).fit()
gwr_results_o.summary()

In [ ]:
for i in range(len(g_X)):
    wi = gwr_results_o.model._build_wi(i, gwr_bw_o).reshape(-1,1)
    x = g_X
    xi_mean = (x*wi).mean()
    y= g_y
    yi_mean = (y*wi).mean()
    numerator = wi*((x - xi_mean) * (y - yi_mean))
    denom = (wi*((np.sqrt((x - xi_mean)**2))) * ((np.sqrt((y - yi_mean)**2))))
    r = (numerator/denom)
    r[np.isnan(r)] = 0

    
r.mean()

In [ ]:
# x_ = np.hstack((x,y))

# a_,_,_ = gwr_results_o.local_collinearity()


# a

In [ ]:
def corr(cov):
    invsd = np.diag(1 / np.sqrt(np.diag(cov)))
    cors = np.dot(np.dot(invsd, cov), invsd)
    return cors

corr_mati = corr(np.cov(x_.T, aweights=wi))